In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Carregar modelo e tokenizador
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-llm-7b-chat")
model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/deepseek-llm-7b-chat",
    device_map="auto",  # Auto-alocação entre dispositivos
    torch_dtype=torch.float16,  # Usa meia precisão
    low_cpu_mem_usage=True  # Reduz uso de RAM
)

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
)

vetorAnimes = FAISS.load_local('C:/Repositorios/RAG-Anime/animes_faiss_index/',embeddings,allow_dangerous_deserialization=True)
vetorEpisodios = FAISS.load_local('C:/Repositorios/RAG-Anime/episodios_faiss_index/',embeddings,allow_dangerous_deserialization=True)
vetorPersonagens = FAISS.load_local('C:/Repositorios/RAG-Anime/personagens_faiss_index/',embeddings,allow_dangerous_deserialization=True)


c:\Anaconda\envs\raganime\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [04:48<00:00, 144.44s/it]
Some parameters are on the meta device because they were offloaded to the cpu.
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_15972\3714368902.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [14]:
# Função para gerar respostas
def gerar_respostaComVetorAnime(query, k=2):
   
    docAnimes = vetorAnimes.similarity_search(query, k=k)

    context1 = "\n".join([doc.page_content for doc in docAnimes])
    

    # Construir prompt
    prompt = f"""
    Com base nas informações abaixo, responda à pergunta:

    Contextos:
    Contexto geral de animes: {context1}

    Pergunta: {query}

    Resposta (use markdown para destacar títulos):
    """
    
    # Gerar resposta 
    inputs = tokenizer(prompt, return_tensors="pt", max_length=2048, truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=200, temperature=0.7)
    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return resposta.split("Resposta:")[-1].strip()

In [15]:
resposta = gerar_respostaComVetorAnime("cowboy bebop")
print(resposta)

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
c:\Anaconda\envs\raganime\Lib\site-packages\transformers\generation\utils.py:2105: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Com base nas informaç�es abaixo, responda à pergunta:

    Contextos:
    Contexto geral de animes: 2nd Season of Gra-P &amp; Rodeo.
.

    Pergunta: cowboy bebop

    Resposta (use markdown para destacar títulos):
    
- **Cowboy Bebop** é um anime japonês, que tem seu 2nd Season de Gra-P & Rodeo. O anime é baseado na série de mangás do mesmo nome, escrita e ilustrada por Hajime Yatate. O anime foi transmitido pela primeira vez na TV Tokyo entre 1998 e 2003, com 26 episódios.

- O enredo gira em torno de três aventureiros, conhecidos como "Cafu, Jet e Faye", que viajam em uma nave espacial chamada "Bebop". A equipe se encontra em miss�es de caça a outros criminosos, enquanto lutam contra seus próprios sentimentos e passados.

- O estilo artístico do anime mescla elementos


In [16]:
# Função para gerar respostas
def gerar_respostaComVetorPersonagens(query, k=2):
   
    docPersonagens = vetorPersonagens.similarity_search(query, k=k)

    context1 = "\n".join([doc.page_content for doc in docPersonagens])
    

    # Construir prompt
    prompt = f"""
    Com base nas informações abaixo, responda à pergunta:

    Contextos:
    Contexto geral de personagens: {context1}

    Pergunta: {query}

    Resposta (use markdown para destacar títulos):
    """
    
    # Gerar resposta 
    inputs = tokenizer(prompt, return_tensors="pt", max_length=2048, truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=200, temperature=0.7)
    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return resposta.split("Resposta:")[-1].strip()

In [17]:
resposta = gerar_respostaComVetorPersonagens("Personagens de cowboy bebop")
print(resposta)

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


Com base nas informaç�es abaixo, responda à pergunta:

    Contextos:
    Contexto geral de personagens: 2nd Season of Gra-P &amp; Rodeo.
.

    Pergunta: Personagens de cowboy bebop

    Resposta (use markdown para destacar títulos):
    
    **Personagens Principais**
    - **Spike Spiegel**: É o protagonista e um dos membros fundadores do Vingadores Bebop. Ele é um ex-assassino e um dos melhores tiros do mundo.
    - **Jet Black**: É o capitão do Vingadores Bebop e o proprietário. Ele é um ex-policial que perdeu a esposa e filho em um acidente.
    - **Ed**: É o cozinheiro do Vingadores Bebop. Ele é um alienígena e tem uma personalidade tímida e obsessiva.
    - **Faye Valentine**: É uma mulher louca e ambiciosa que se junta ao Vingadores Bebop como uma nova membro. Ela tem habilidades de luta e é conhecida como a "Rainha dos Roubos".

**Person


In [18]:
# Função para gerar respostas
def gerar_respostaComVetorEpisodios(query, k=2):
   
    docEpis = vetorEpisodios.similarity_search(query, k=k)

    context1 = "\n".join([doc.page_content for doc in docEpis])
    

    # Construir prompt
    prompt = f"""
    Com base nas informações abaixo, responda à pergunta:

    Contextos:
    Contexto geral de episódios: {context1}

    Pergunta: {query}

    Resposta (use markdown para destacar títulos):
    """
    
    # Gerar resposta 
    inputs = tokenizer(prompt, return_tensors="pt", max_length=2048, truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=200, temperature=0.7)
    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return resposta.split("Resposta:")[-1].strip()

In [ ]:
resposta = gerar_respostaComVetorEpisodios("cowboy bebop")
print(resposta)

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


Com base nas informaç�es abaixo, responda à pergunta:

    Contextos:
    Contexto geral de episódios: As Tasmania Kid and Bighorn wander through a field beside the Yukikaze, Kid plucks a flower from the ground and shows it to Bighorn, commenting that he thinks the whole field is a bit on the gaudy side. The red hue of flower sends the pent-up Bighorn into a rage, and he repeatedly headbutts a rock wall as he roars about the current lack of action. Meanwhile, aboard the ship, Apache and NAVI report to Lio Convoy that they are unable to get a fix on the Destrons' ship. Lio Convoy arranges for an on-foot search of the planet, and Scuba and Diver go outside to inform Kid and Bighorn. Bighorn roars with delight and immediately stampedes off, followed by Kid; Diver turns to suggest that he and Scuba go in the opposite direction, but Scuba has already gone off on his own. Elsewhere, aboard the Galvaburg II, with Galvatron still unconscious after his experience with the ancient Gaean computer